In [ ]:
__author__ = 'Knut Olsen <knut.olsen@noirlab.edu>, Robert Nikutta <robert.nikutta@noirlab.edu>'
__version__ = '20231029' # yyyymmdd; version datestamp of this notebook
__datasets__ = ['smash_dr1', 'des_dr1']
__keywords__ = ['tutorial', 'query', 'image cutout', 'globular cluster', 'SIA']

# Welcome to Astro Data Lab
*Knut Olsen, Robert Nikutta, Astro Data Lab Team*

## Table of contents
* [Goals](#goals)
* [Summary](#summary)
* [Disclaimer & attribution](#attribution)
* [Imports & setup](#import)
* [Authentication](#auth)
* [Basic info about database tables](#basic)
* [A simple database query](#query)
* [An image cutout](#image)
* [Resources and references](#resources)

<a class="anchor" id="goals"></a>
# Goals
Learn how to:
* Import standard Data Lab modules
* Set up the Simple Image Access (SIA) Service to let you create image cutouts
* Discover the datasets available in the database
* Issue a simple query to the database
* Retrieve image cutouts
* Create a color image


<a class="anchor" id="summary"></a>
# Summary

If you've gotten this far, you're planning to use the Data Lab Jupyter notebook environment to access, explore, and analyze datasets available in the Data Lab.  This notebook aims to provide you with a very quick overview of how to use some common Data Lab services.  For more detailed explanations of data access, the full range of services, or for complete science examples, check out the other notebooks in this directory tree.

<a class="anchor" id="attribution"></a>
# Disclaimer & attribution
If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, http://dx.doi.org/10.1117/12.2057445

* Data Lab disclaimer: https://datalab.noirlab.edu/disclaimers.php

<a class="anchor" id="import"></a>
# Imports and setup
To use the Astro Data Lab, you'll generally want to import common packages such as NumPy and matplotlib.  From the datalab package, you'll at minimum need the authClient module to get an authorization token (even if using Data Lab anonymously) and the queryClient module to issue a query against the catalog database.

For storing results in virtual storage and myDB, you'll need the storeClient module *and* log in as an authenticated user.  

The helpers module has many convience functions.  See the <a href="http://datalab.noirlab.edu/docs/manual/UsingAstroDataLab/ClientInterfaces/Helpers/Helpers.html">online documentation</a> for a summary.

Use of the image cutout service (SIA) requires the external PyVO package, and the URL of the SIA service that you will use.  Data Lab has a general SIA service containing all available images from [Astro Data Archive](https://astroarchive.noirlab.edu/), as well as a number of survey-specific services.  See the [SIA service HowTo notebook](https://github.com/astro-datalab/notebooks-latest/blob/master/04_HowTos/SiaService/How_to_use_the_Simple_Image_Access_service.ipynb) for examples.

In [ ]:
# std library imports
from getpass import getpass

# 3rd party
import numpy as np
import pylab as plt
import matplotlib
from astropy import utils, io
from astropy.visualization import make_lupton_rgb
%matplotlib inline

# Data Lab and related imports

# You'll need at least these for authenticating and for issuing database queries
from dl import authClient as ac, queryClient as qc

# Get helpers for various convenience function
from dl.helpers.utils import convert

# You'll want storeClient if you plan to use virtual storage or myDB
# from dl import storeClient as sc

# To get image cutouts, you'll need the VO-based SIA package, and define which SIA service to use
from pyvo.dal import sia
DEF_ACCESS_URL = "https://datalab.noirlab.edu/sia/des_dr1" # DES SIA service URL
svc = sia.SIAService(DEF_ACCESS_URL)

<a class="anchor" id="auth"></a>
# Authentication
For the purposes of this notebook, there is no need to log in with your username and password inside the notebook. As an anonymous user, you can issue queries to the database or retrieve image cutouts, but not store your results in virtual storage or myDB. If you need these things, you would use the authClient module to log in. You only need to do this once (unless you log out through authClient), as the authentication token is stored on the server and automatically detected.

If you need to log in to Data Lab, un-comment the cell below and execute it:

In [ ]:
#token = ac.login(input("Enter user name: (+ENTER) "),getpass("Enter password: (+ENTER) "))
ac.whoAmI()

<a class="anchor" id="basic"></a>
# Basic info about database tables

### What datasets are available?
The queryClient has a `schema` method to give you information about available databases, tables, and columns.  If we call `qc.schema()` with an empty first argument, we'll get information on the available datasets and a one-line description for most of them.

In [ ]:
print(qc.schema())

### Get list of tables
If we call `qc.schema()` with a specific dataset name, we'll see what tables are available for that dataset.  Here's what's available for SMASH DR1:

In [ ]:
print(qc.schema('smash_dr1'))

### Get list of columns
We can also use `qc.schema()` to get column names and descriptions for a specific table.  Here's what's available for the SMASH DR1 object table.  (Note that not all datasets have column descriptions for every column).

In [ ]:
print(qc.schema('smash_dr1.object'))

### Getting statistics for tables
You'll often want to get some basic information about a given table, e.g. the number of rows.  One might be tempted to use `SELECT COUNT(*)` in a query for this -- but that can be slow on large datasets, and is often not needed. Instead, the special database `tbl_stat` contains this information for each dataset. Let's query this table instead:

In [ ]:
#query = "SELECT COUNT(ra) FROM phat_v2.phot_mod" # SLOW
query = "SELECT * FROM tbl_stat WHERE schema='smash_dr1' AND tbl_name='object'" # Retrieve useful stats, quickly

In [ ]:
info = qc.query(sql=query) # by default the result is a CSV formatted string

In [ ]:
print(info)

<a class="anchor" id="query"></a>
# A simple query for catalog data
In the above section we already saw a basic query of the `tbl_stats` database. Here we will retrieve the first 10 rows from the `smash_dr1.object` table:

In [ ]:
query = "SELECT * FROM smash_dr1.object LIMIT 10"
result = qc.query(sql=query) # by default the result is a CSV formatted string

Note that by default the result is returned as a long string that looks like a CSV file:

In [ ]:
print(type(result))
print(result[:1000]) # print just the first 1000 characters of the returned string

We would generally want to convert this string into a table or an array, i.e., into some format that allows numerical processing.  The Data Lab `helpers` module makes it easy. Here we convert the result string result into a Pandas dataframe:

In [ ]:
df = convert(result,'pandas')
df.head()

Note that for convenience one can request the result to be in a number of other formats, including a Pandas DataFrame: 

In [ ]:
query = "SELECT * FROM smash_dr1.object LIMIT 10"
df = qc.query(sql=query,fmt='pandas')
df.head()

The results of a query like this can be stored to your VOSpace (remote user file storage) or to MyDB (remote user database), if you logged in as an authenticated user. See some of the Data Lab [How-To notebooks](https://github.com/astro-datalab/notebooks-latest/tree/058d6a71dbdd7f4f19a38f5ec6f78cd9b107113c/04_HowTos) for for examples of this.

<a class="anchor" id="image"></a>
# Find and download an image cutout
Using the image cutout service is a two-step process. First, we specify a position (RA and Dec), and the size of the image that we want to retrieve. Second, we then search the SIA service for all images that overlap that point on the sky:

In [ ]:
# NGC 288 (a globular cluster)
ra = 13.19     # in decimal degrees
dec = -26.59   # in decimal degrees

fov = 13/60  # image cutout field of view (in degrees; here 13 arminutes = 0.22 degrees)

imgTable = svc.search((ra,dec), (fov/np.cos(dec*np.pi/180), fov), verbosity=2).to_table()  # uses declination correction

The result is a VOTable that we convert to an astropy Table on the fly. The table contains many columns of metadata describing the parameters of each image, including a URL for the cutout itself. Note, however, that data quality images such as masks or weight maps can also appear in the list:

In [ ]:
print(type(imgTable))
imgTable

The next step is to identify the image that you want from the list of available images.  Here we will limit the list to g-band image stacks, and select the `image` product type (rather than weights or masks).

In [ ]:
# boolean selection for just g-band images
sel0 = imgTable['obs_bandpass'] == 'g'

# logically add selections for 'Stack' and 'image' product type
sel = sel0 & ((imgTable['proctype'] == 'Stack') & (imgTable['prodtype'] == 'image')) # basic selection

# filter the above results table
Table = imgTable[sel] # select
Table

The final step is to extract the URL and download the image:

In [ ]:
row = Table[0]
url = row['access_url'] # get the download URL
print(url)

In [ ]:
gimage = io.fits.getdata(utils.data.download_file(url,cache=True,show_progress=False,timeout=120))

We'll do a quick display here:

In [ ]:
fig = plt.subplots(figsize=(10,10))
plt.imshow(np.arcsinh(gimage),cmap='gray',norm=matplotlib.colors.Normalize(vmin=0))
plt.axis('off');

## Let's make a color image
Let's write a quick function to do the SIA query and image selection for us:

In [ ]:
# A function to download the deepest available stacked images
def download_deep_stack(ra,dec,fov=0.1,band='g'):
    imgTable = svc.search((ra,dec), (fov/np.cos(dec*np.pi/180), fov), verbosity=2).to_table()
    print("The full image list contains", len(imgTable), "entries")

    # apply image seletion criteria
    sel0 = imgTable['obs_bandpass'] == band
    sel = sel0 & ((imgTable['proctype'] == 'Stack') & (imgTable['prodtype'] == 'image')) # basic selection
    Table = imgTable[sel] # select

    # if more than one image matches our desired criteria
    if (len(Table)>0):
        row = Table[np.argmax(Table['exptime'].data.data.astype('float'))] # pick image with longest exposure time
        url = row['access_url'] # get the download URL
        print ('Downloading deepest stacked image...')
        image = io.fits.getdata(utils.data.download_file(url,cache=True,show_progress=False,timeout=120))
        print(url)

    # if no images match our criteria
    else:
        print ('No image available.')
        image = None
        
    print()
    return image

Using this function we can now quickly download two more bands (r and i bands), and combine them with the g-band image to make a false-color image:

In [ ]:
rimage = download_deep_stack(ra,dec,fov,band='r')
iimage = download_deep_stack(ra,dec,fov,band='i')

Now we can do the combination of images in three bands:

In [ ]:
color_image = make_lupton_rgb(iimage, rimage, gimage, stretch=100, Q=0.1)

And plot the resulting false-color image:

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.imshow(color_image)
plt.axis('off');

<a class="anchor" id="resources"></a>
# Further resources and references
* This notebook is [available on GitHub](https://github.com/astro-datalab/notebooks-latest/blob/058d6a71dbdd7f4f19a38f5ec6f78cd9b107113c/01_GettingStartedWithDataLab/02_GettingStartedWithDataLab.ipynb)
* An in-depth [notebook on Simple Image Access](https://github.com/astro-datalab/notebooks-latest/blob/058d6a71dbdd7f4f19a38f5ec6f78cd9b107113c/04_HowTos/SiaService/How_to_use_the_Simple_Image_Access_service.ipynb)
* Info about the [globular cluster NGC 288](https://en.wikipedia.org/wiki/NGC_288)